In [ ]:
# Autoreaload and show all outputs
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Show all outputs for each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pandas as pd
from pathlib import Path
from dataclasses import dataclass

round_number = 464
global_folder = Path('/home/pica/hdd/nas/Data/numerai')
data_folder =Path(f'/home/pica/nas_pica/Data/numerai/data/numerai_dataset_{round_number}')

@dataclass
class Version:
    name: str
    folder: Path
    round_number: int

poetry = Version(name='poetry', folder=global_folder / f'submissions', round_number=round_number)
automl = Version(name='automl', folder=global_folder / f'submissions_bak_{round_number}_automl', round_number=round_number)
versions = [poetry, automl]

model_name = 'rpica_4'
model_name = 'rpica'
model_name = 'rpica_test_2'
model_name = 'rpica_test_1'
submission_relative = f'{model_name}/{model_name}_submission_{round_number}.csv'

round_df = pd.read_csv(data_folder/'numerai_tournament_data.csv')

round_df['data_type'].value_counts()
is_live = round_df.set_index('id')['data_type'] == 'live'

for version in versions:
    path = version.folder / submission_relative
    df = pd.read_csv(path).set_index('id')[is_live]
    df['rank'] = df['prediction'].rank()
    version.df = df
    print(f'For {version.name}, shape: {version.df.shape}')

df = pd.merge(poetry.df,
              automl.df,
              on='id',
              suffixes=['_poetry', '_automl'])
df['diff'] = poetry.df['prediction'] != automl.df['prediction']

In [ ]:
# Con rpica_test_1 el rank es igual al 95.2%
# Con rpica_test_2 el rank es igual al 100.0%
# Con rpica_4 el rank es igual al 100.0%
# Con rpica_1 el rank es igual al 100.0%

rank_equal = df['rank_poetry'] == df['rank_automl']
rank_equal.sum()/len(rank_equal)
df[~rank_equal].head()

## Mirar lo de rows live

In [ ]:
import pandas as pd 
from pathlib import Path
folder = Path('/home/pica/nas_pica/Data/numerai/data/numerai_dataset_458')

tournament_path = folder/'numerai_tournament_data.csv'
train_path = folder/'numerai_training_data.csv'

tournament_df = pd.read_csv(tournament_path)
train_df = pd.read_csv(train_path)

In [ ]:
for name, df in zip(['tournament', 'train'], [tournament_df, train_df]):
    print(name)
    print(df.shape)
    feat_cols = [col for col in df.columns if col.startswith('feature')]
    non_feat_cols = [col for col in df.columns if not col.startswith('feature')]
    print(non_feat_cols)
    print(df['data_type'].value_counts())

    print()

In [ ]:
df = tournament_df
df['data_type'].value_counts()
df['target'].value_counts(dropna=False)
df['target'].value_counts().sum()